In [1]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils import shuffle

## Train same model as previous notebook

In [3]:
# Load simulated data: load complete training and validation sets.
np.random.seed(11)

# Training set should be one concatenated shuffled list of simulated examples
# Validation set similarly, but also keep information about simulation parameters
# to evaluate performance in specific simulation conditions.
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
y, x = shuffle(
    np.concatenate([np.loadtxt(bounds_file) for bounds_file in bounds_files]),
    np.concatenate([np.loadtxt(intensities_file) for intensities_file in intensities_files])
)

In [4]:
def overlap_metric(y_true, y_predicted):
    y_pred_round = tf.round(y_predicted)
    max_start = tf.math.maximum(y_true[:, 0], y_pred_round[:, 0])
    min_end = tf.math.minimum(y_true[:, 1], y_pred_round[:,1])
    overlap = tf.math.maximum(0.0, min_end - max_start)
    longest_bounds = tf.math.maximum(
        y_true[:, 1] + 1 - y_true[:, 0], 
        y_pred_round[:, 1] + 1 - y_pred_round[:, 0]
        )
    return overlap / longest_bounds

In [5]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='linear')
    ])
model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss='mse', 
    metrics=['mae', overlap_metric]
    )

In [9]:
history = model.fit(x, y, epochs=1000, batch_size=8192, validation_split=0.33)

Train on 60299 samples, validate on 29701 samples
Epoch 1/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.1221 - mean_absolute_error: 1.3484 - overlap_metric: 0.5780 - val_loss: 3.0423 - val_mean_absolute_error: 1.3340 - val_overlap_metric: 0.5976
Epoch 2/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.8640 - mean_absolute_error: 1.2782 - overlap_metric: 0.5896 - val_loss: 2.7520 - val_mean_absolute_error: 1.2420 - val_overlap_metric: 0.5712
Epoch 3/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5329 - mean_absolute_error: 1.1917 - overlap_metric: 0.6035 - val_loss: 2.4720 - val_mean_absolute_error: 1.1732 - val_overlap_metric: 0.6132
Epoch 4/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5537 - mean_absolute_error: 1.1991 - overlap_metric: 0.6050 - val_loss: 2.5686 - val_mean_absolute_error: 1.2035 - val_overlap_metric: 0.6181
Epoch 5/1000
60299/60299 [============================

60299/60299 [==============================] - 0s 1us/step - loss: 2.5023 - mean_absolute_error: 1.1881 - overlap_metric: 0.6097 - val_loss: 2.4472 - val_mean_absolute_error: 1.1620 - val_overlap_metric: 0.6092
Epoch 38/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.4508 - mean_absolute_error: 1.1722 - overlap_metric: 0.6105 - val_loss: 2.4621 - val_mean_absolute_error: 1.1726 - val_overlap_metric: 0.6137
Epoch 39/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5098 - mean_absolute_error: 1.1891 - overlap_metric: 0.6081 - val_loss: 2.4445 - val_mean_absolute_error: 1.1653 - val_overlap_metric: 0.6134
Epoch 40/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.4353 - mean_absolute_error: 1.1687 - overlap_metric: 0.6122 - val_loss: 2.8665 - val_mean_absolute_error: 1.2873 - val_overlap_metric: 0.6047
Epoch 41/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.6797 - mean_absolute_error: 1.2

60299/60299 [==============================] - 0s 1us/step - loss: 2.6210 - mean_absolute_error: 1.2246 - overlap_metric: 0.6020 - val_loss: 2.3745 - val_mean_absolute_error: 1.1452 - val_overlap_metric: 0.6145
Epoch 74/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5560 - mean_absolute_error: 1.2055 - overlap_metric: 0.6059 - val_loss: 2.7628 - val_mean_absolute_error: 1.2698 - val_overlap_metric: 0.6029
Epoch 75/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.4016 - mean_absolute_error: 1.1591 - overlap_metric: 0.6134 - val_loss: 2.4084 - val_mean_absolute_error: 1.1598 - val_overlap_metric: 0.6096
Epoch 76/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.5441 - mean_absolute_error: 1.4462 - overlap_metric: 0.5626 - val_loss: 2.4664 - val_mean_absolute_error: 1.1743 - val_overlap_metric: 0.6166
Epoch 77/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5694 - mean_absolute_error: 1.2

60299/60299 [==============================] - 0s 1us/step - loss: 2.4569 - mean_absolute_error: 1.1743 - overlap_metric: 0.6105 - val_loss: 2.3367 - val_mean_absolute_error: 1.1364 - val_overlap_metric: 0.6214
Epoch 110/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.3260 - mean_absolute_error: 1.1368 - overlap_metric: 0.6167 - val_loss: 2.3639 - val_mean_absolute_error: 1.1484 - val_overlap_metric: 0.6120
Epoch 111/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.2708 - mean_absolute_error: 1.1208 - overlap_metric: 0.6192 - val_loss: 2.8052 - val_mean_absolute_error: 1.2827 - val_overlap_metric: 0.5992
Epoch 112/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.6703 - mean_absolute_error: 1.4983 - overlap_metric: 0.5597 - val_loss: 3.2626 - val_mean_absolute_error: 1.3878 - val_overlap_metric: 0.5685
Epoch 113/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.6905 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.5467 - mean_absolute_error: 1.2023 - overlap_metric: 0.6074 - val_loss: 3.6737 - val_mean_absolute_error: 1.4952 - val_overlap_metric: 0.5805
Epoch 146/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9258 - mean_absolute_error: 1.3010 - overlap_metric: 0.5895 - val_loss: 2.3504 - val_mean_absolute_error: 1.1403 - val_overlap_metric: 0.6237
Epoch 147/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.3897 - mean_absolute_error: 1.1563 - overlap_metric: 0.6128 - val_loss: 2.4849 - val_mean_absolute_error: 1.1776 - val_overlap_metric: 0.6134
Epoch 148/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.3002 - mean_absolute_error: 1.1338 - overlap_metric: 0.6166 - val_loss: 2.2987 - val_mean_absolute_error: 1.1223 - val_overlap_metric: 0.6205
Epoch 149/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.2402 - mean_absolute_error:

...

60299/60299 [==============================] - 0s 1us/step - loss: 1.7403 - mean_absolute_error: 0.9543 - overlap_metric: 0.6472 - val_loss: 1.8012 - val_mean_absolute_error: 0.9719 - val_overlap_metric: 0.6470
Epoch 830/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7348 - mean_absolute_error: 0.9520 - overlap_metric: 0.6480 - val_loss: 1.7719 - val_mean_absolute_error: 0.9600 - val_overlap_metric: 0.6513
Epoch 831/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7165 - mean_absolute_error: 0.9454 - overlap_metric: 0.6489 - val_loss: 1.7388 - val_mean_absolute_error: 0.9453 - val_overlap_metric: 0.6516
Epoch 832/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.6881 - mean_absolute_error: 0.9355 - overlap_metric: 0.6505 - val_loss: 1.7680 - val_mean_absolute_error: 0.9621 - val_overlap_metric: 0.6477
Epoch 833/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7235 - mean_absolute_error:

60299/60299 [==============================] - 0s 2us/step - loss: 1.7325 - mean_absolute_error: 0.9547 - overlap_metric: 0.6472 - val_loss: 1.7255 - val_mean_absolute_error: 0.9478 - val_overlap_metric: 0.6495
Epoch 866/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.6602 - mean_absolute_error: 0.9264 - overlap_metric: 0.6521 - val_loss: 1.6990 - val_mean_absolute_error: 0.9332 - val_overlap_metric: 0.6516
Epoch 867/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.6998 - mean_absolute_error: 0.9433 - overlap_metric: 0.6491 - val_loss: 1.8041 - val_mean_absolute_error: 0.9647 - val_overlap_metric: 0.6398
Epoch 868/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.7121 - mean_absolute_error: 0.9464 - overlap_metric: 0.6478 - val_loss: 1.6853 - val_mean_absolute_error: 0.9277 - val_overlap_metric: 0.6521
Epoch 869/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.7711 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 1.6309 - mean_absolute_error: 0.9168 - overlap_metric: 0.6535 - val_loss: 1.7427 - val_mean_absolute_error: 0.9477 - val_overlap_metric: 0.6449
Epoch 902/1000
60299/60299 [==============================] - 0s 1us/step - loss: 4.2377 - mean_absolute_error: 1.5970 - overlap_metric: 0.5382 - val_loss: 4.0339 - val_mean_absolute_error: 1.6017 - val_overlap_metric: 0.5636
Epoch 903/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.7300 - mean_absolute_error: 1.5060 - overlap_metric: 0.5604 - val_loss: 3.5991 - val_mean_absolute_error: 1.4518 - val_overlap_metric: 0.5501
Epoch 904/1000
60299/60299 [==============================] - 0s 2us/step - loss: 2.7565 - mean_absolute_error: 1.2452 - overlap_metric: 0.5959 - val_loss: 2.6373 - val_mean_absolute_error: 1.2152 - val_overlap_metric: 0.5988
Epoch 905/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.2139 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 1.7185 - mean_absolute_error: 0.9527 - overlap_metric: 0.6477 - val_loss: 1.8715 - val_mean_absolute_error: 1.0031 - val_overlap_metric: 0.6478
Epoch 938/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.8687 - mean_absolute_error: 1.0082 - overlap_metric: 0.6383 - val_loss: 1.7518 - val_mean_absolute_error: 0.9562 - val_overlap_metric: 0.6534
Epoch 939/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7697 - mean_absolute_error: 0.9693 - overlap_metric: 0.6446 - val_loss: 1.9532 - val_mean_absolute_error: 1.0232 - val_overlap_metric: 0.6349
Epoch 940/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7952 - mean_absolute_error: 0.9796 - overlap_metric: 0.6430 - val_loss: 1.9062 - val_mean_absolute_error: 1.0231 - val_overlap_metric: 0.6427
Epoch 941/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.7403 - mean_absolute_error:

60299/60299 [==============================] - 0s 2us/step - loss: 1.6673 - mean_absolute_error: 0.9282 - overlap_metric: 0.6514 - val_loss: 1.6937 - val_mean_absolute_error: 0.9322 - val_overlap_metric: 0.6575
Epoch 974/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.6350 - mean_absolute_error: 0.9161 - overlap_metric: 0.6543 - val_loss: 1.6512 - val_mean_absolute_error: 0.9114 - val_overlap_metric: 0.6546
Epoch 975/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.6492 - mean_absolute_error: 0.9233 - overlap_metric: 0.6529 - val_loss: 1.6400 - val_mean_absolute_error: 0.9106 - val_overlap_metric: 0.6558
Epoch 976/1000
60299/60299 [==============================] - 0s 2us/step - loss: 1.8954 - mean_absolute_error: 1.0146 - overlap_metric: 0.6379 - val_loss: 2.2240 - val_mean_absolute_error: 1.1122 - val_overlap_metric: 0.6103
Epoch 977/1000
60299/60299 [==============================] - 0s 1us/step - loss: 1.8781 - mean_absolute_error:

Check same metrics as previous notebook.

In [10]:
print("Mean absolute error", history.history['val_mean_absolute_error'][-1])
print("Overlap metric", history.history['val_overlap_metric'][-1])

Mean absolute error 0.9403871890210266
Overlap metric 0.6472789578307603


## Validation on different datasets

In [11]:
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
